In [2]:
%matplotlib inline
#%load_ext autoreload
#%autoreload 2
%reload_ext autoreload
import numpy as np
import matplotlib.pyplot as plt
import math, sys, os
from numpy.random import randn
from sklearn.datasets import make_blobs

# setup pyspark for IPython_notebooks
spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.8.2.1-src.zip'))
execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.6.0
      /_/

Using Python version 2.7.10 (default, Jul 13 2015 12:05:58)
SparkContext available as sc, HiveContext available as sqlContext.


In [31]:
data_home = os.environ.get('DATA_HOME', None)
sys.path.insert(0, data_home)

# data
from gen_data import make_blobs_rdd

# utilitiy functions for this notebook
from lsh_util import *

# make some data
N = 1000
d = 2
k = 3
sigma = 1
bound = 10*sigma

data_RDD = make_blobs_rdd(N, d, k, sigma, bound, sc)
data_RDD.take(2)

[(0, array([-0.78571077, -0.42775112])),
 (2, array([-0.97998542, -0.88874468]))]

# Stable Distribution Projection Hashing

In [32]:
def config_stable_dist_proj(d, p = 5, r = 2.0, seed = None):
    # random projection vectors
    A = np.random.multivariate_normal(np.zeros(d), np.eye(d), p)
    B = np.random.rand(1,p)
    def projection_func(tup):
        y, x = tup # expect key (int, 1xD vector)
        projs = ((A.dot(x) / r) + B).flatten()
        bucket = to_bucket(projs)
        
        return (bucket, y)
    
    return (A, B, projection_func)

In [33]:
A, B, hash_func = config_stable_dist_proj(d)

gini_impurities = data_RDD.map(hash_func).map(to_dict).reduceByKey(reduce_count_clusters).map(gini_impurity).collect()
for b, g, c in sorted(gini_impurities):
    print "bucket: %s , in bucket: %d , gini_impurity: %f" % (b, c, g)


bucket: 27 , in bucket: 226 , gini_impurity: 0.478463
bucket: 29 , in bucket: 51 , gini_impurity: 0.398308
bucket: 31 , in bucket: 723 , gini_impurity: 0.656264


In [34]:
# impurity as we scale up the number of hyperplanes used for projections
for n_Z in range(10,201,10):
    A, B, hash_func = config_stable_dist_proj(d, n_Z)
    gini_impurities = data_RDD.map(hash_func).map(to_dict).reduceByKey(reduce_count_clusters).map(gini_impurity).collect()
    g_i = weighted_gini(gini_impurities)
    print "%d projections, gini_impurity: %f" % (n_Z, g_i)
    

10 projections, gini_impurity: 0.117820
20 projections, gini_impurity: 0.075784
30 projections, gini_impurity: 0.129946
40 projections, gini_impurity: 0.239150
50 projections, gini_impurity: 0.049222
60 projections, gini_impurity: 0.046699
70 projections, gini_impurity: 0.027854
80 projections, gini_impurity: 0.052600
90 projections, gini_impurity: 0.025314
100 projections, gini_impurity: 0.028558
110 projections, gini_impurity: 0.036383
120 projections, gini_impurity: 0.025243
130 projections, gini_impurity: 0.025483
140 projections, gini_impurity: 0.024040
150 projections, gini_impurity: 0.023013
160 projections, gini_impurity: 0.016629
170 projections, gini_impurity: 0.019714
180 projections, gini_impurity: 0.014907
190 projections, gini_impurity: 0.014920
200 projections, gini_impurity: 0.014035
